In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [35]:
h_data = np.genfromtxt('output/hub1d_Ni8_Ne4/soft_hamiltonian_U0.0.txt')
h_data_r = h_data[:, :2].astype('int')
h_data_v = h_data[:, 2:]

Nb = np.max(h_data_r[:, 0]) + 1
h = np.zeros((Nb, Nb), dtype='complex')

for r, v in zip(h_data_r, h_data_v):
    h[r[0], r[1]] = v[0] + 1j*v[1]
print(h.real)

eigval, eigvec = np.linalg.eig(h)
print(eigval)
print(eigvec)

[[ 0.  -1.   0.   0.   0.   0.   0.   1. ]
 [-1.   0.1 -1.   0.   0.   0.   0.   0. ]
 [ 0.  -1.   0.2 -1.   0.   0.   0.   0. ]
 [ 0.   0.  -1.   0.3 -1.   0.   0.   0. ]
 [ 0.   0.   0.  -1.   0.4 -1.   0.   0. ]
 [ 0.   0.   0.   0.  -1.   0.5 -1.   0. ]
 [ 0.   0.   0.   0.   0.  -1.   0.6 -1. ]
 [ 1.   0.   0.   0.   0.   0.  -1.   0.7]]
[-1.65120688+0.j -1.39395013+0.j  2.35120688+0.j  2.09395013+0.j
  1.05658169+0.j  1.16520014+0.j -0.46520014+0.j -0.35658169+0.j]
[[ 0.47866156-0.j -0.3255628 +0.j -0.23856133+0.j -0.3492342 +0.j
   0.47687609+0.j -0.09839913+0.j  0.48860814+0.j  0.10091992+0.j]
 [ 0.55180793+0.j -0.10458411+0.j  0.08224549+0.j  0.40571619+0.j
  -0.40293863+0.j -0.37395346+0.j  0.12890145+0.j -0.4408899 +0.j]
 [ 0.48766829-0.j  0.16931936+0.j  0.05340973+0.j -0.45974365+0.j
  -0.09143237+0.j  0.49673441+0.j -0.41575302+0.j -0.30222217+0.j]
 [ 0.35096696-0.j  0.37447072+0.j -0.19714086+0.j  0.46501536+0.j
   0.48125793+0.j -0.10549466+0.j -0.40546042+0.j  0.272678